<a href="https://colab.research.google.com/github/minhaz1172/NLP/blob/main/Text_Classification(Using_ML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We will be using a dataset called "Economic news article tone and relevance" which consists of approximately 8000 news articles, which were tagged as relevant or not relevant to the US Economy. Our goal in this kernel is to explore the process of training and testing text classifiers for this dataset.

In [1]:
import numpy as np
import pandas as pd

In [2]:

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minhaz1172","key":"c20c43ca9e6e5be6fa69c912fab503c3"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!pip install kaggle

In [7]:
!kaggle datasets download -d heeraldedhia/us-economic-news-articles
!unzip us-economic-news-articles.zip -d data


Dataset URL: https://www.kaggle.com/datasets/heeraldedhia/us-economic-news-articles
License(s): CC0-1.0
us-economic-news-articles.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  us-economic-news-articles.zip
replace data/US-Economic-News.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [11]:
df=pd.read_csv("data/US-Economic-News.csv", encoding='latin-1')
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


# separating target and feature

In [21]:
data = df.drop(['_unit_id','_golden', '_unit_state','_trusted_judgments','_last_judgment_at','positivity','positivity:confidence','relevance:confidence',	'articleid','date','headline','positivity_gold','relevance_gold'], axis=1)
data.head()

,relevance,text
0,yes,NEW YORK -- Yields on most certificates of dep...
1,no,The Wall Street Journal Online</br></br>The Mo...
2,no,WASHINGTON -- In an effort to achieve banking ...
3,no,The statistics on the enormous costs of employ...
4,yes,NEW YORK -- Indecision marked the dollar's ton...


## swapping text and relevance

In [23]:
data = data[['text', 'relevance']]
data.head()

,text,relevance
0,NEW YORK -- Yields on most certificates of dep...,yes
1,The Wall Street Journal Online</br></br>The Mo...,no
2,WASHINGTON -- In an effort to achieve banking ...,no
3,The statistics on the enormous costs of employ...,no
4,NEW YORK -- Indecision marked the dollar's ton...,yes


In [28]:
data.shape

(8000, 2)

In [24]:
data['text'][1]

'The Wall Street Journal Online</br></br>The Morning Brief, a look at the day\'s biggest news, is emailed to subscribers by 7 a.m. every business day. Sign up for the e-mail here.</br></br>On Friday evening, with Congress out of town on its summer recess and Americans heading into a mid-August weekend, the Bush administration sent a message to the states: The federal government will make it tougher for a national children\'s insurance program to cover the offspring of middle-income families.</br></br>The State Children\'s Health Insurance Program was created in 1997 to help children whose families couldn\'t afford insurance but didn\'t qualify for Medicaid, and administration officials tell the New York Times that the changes are aimed at returning the program to its low- income focus and assuring it didn\'t become a replacement for private insurance. Administration point man Dennis Smith wrote to state officials saying there would be new restrictions on the District of Columbia and th

In [30]:
data['relevance'].value_counts()

,count
relevance,
no,6571
yes,1420
not sure,9


In [27]:
data.isnull().sum()

,0
text,0
relevance,0


# plot a bar

In [32]:
import plotly.graph_objects as go
fig=go.Figure(go.Bar(x=data['relevance'].value_counts().index, y=data['relevance'].value_counts()))
# Customize the layout
fig.update_layout(
    title='Relevance vs Sentiment',
    xaxis_title='Relevance',
    yaxis_title='Number of SEntiments',
    template=''  # Use a specific theme
)
fig.show()